In [2]:
# ! git clone --single-branch --branch feature/add_transformers https://github.com/OFA-Sys/OFA.git
# ! pip install OFA/transformers/
# ! git clone https://huggingface.co/OFA-Sys/OFA-large

# ! git clone https://github.com/claytonbonelli/sequence_generator.git
# ! pip install sequence_generator/


Cloning into 'OFA'...
remote: Enumerating objects: 5745, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 5745 (delta 29), reused 25 (delta 25), pack-reused 5714
Receiving objects: 100% (5745/5745), 97.78 MiB | 551.00 KiB/s, done.
Resolving deltas: 100% (2215/2215), done.
Updating files: 100% (3223/3223), done.
Looking in indexes: https://repo.huaweicloud.com/repository/pypi/simple
Processing ./OFA/transformers
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done

In [2]:
from tqdm import tqdm
from PIL import Image
from transformers import OFATokenizer, OFAModel
from torch.utils.data import DataLoader

from PIL import Image
from torchvision import transforms
from transformers import OFATokenizer, OFAModel
import sequence_generator
import torch

In [3]:


mean, std = [0.5, 0.5, 0.5], [0.5, 0.5, 0.5]
resolution = 480
patch_resize_transform = transforms.Compose([
        lambda image: image.convert("RGB"),
        transforms.Resize((resolution, resolution), interpolation=Image.BICUBIC),
        transforms.ToTensor(), 
        transforms.Normalize(mean=mean, std=std)
    ])

ckpt_dir = "OFA-Sys/ofa-large"
tokenizer = OFATokenizer.from_pretrained(ckpt_dir)

txt = " what does the image describe?"
inputs = tokenizer([txt], return_tensors="pt").input_ids
img = Image.open("images/92e911621.png")
patch_img = patch_resize_transform(img).unsqueeze(0)


model = OFAModel.from_pretrained(ckpt_dir, use_cache=True)
generator = sequence_generator.SequenceGenerator(
        tokenizer=tokenizer,
        beam_size=5,
        max_len_b=16, 
        min_len=0,
        no_repeat_ngram_size=3,
    )
data = {}
data["net_input"] = {"input_ids": inputs, 'patch_images': patch_img, 'patch_masks':torch.tensor([True])}
gen_output = generator.generate([model], data)
gen = [gen_output[i][0]["tokens"] for i in range(len(gen_output))]

model = OFAModel.from_pretrained(ckpt_dir, use_cache=False)
gen = model.generate(inputs, patch_images=patch_img, num_beams=5, no_repeat_ngram_size=3) 

print(tokenizer.batch_decode(gen, skip_special_tokens=True))


OFA-Sys/ofa-large
<super: <class 'OFATokenizer'>, <OFATokenizer object>>


Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

KeyboardInterrupt: 